In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [8]:
ad = pd.read_csv('adFeature.csv')
user = pd.read_csv('userFeature.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test1.csv')

ad.shape
ad.head()

user.shape
user.head()

train.shape
train.head()

test.shape
test.head()

(173, 8)

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,8203,76104,1500666,59,282,0,6
1,2050,19441,178687,245165,53,1,0,6
2,1716,5552,158101,1080850,35,27,113,9
3,336,370,4833,119845,22,67,113,9
4,671,45705,352827,660519,42,67,0,4


(11420039, 24)

,uid,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,26325489,4,2,11,7,2,950.0,93;70;77;86;109;47;75;69;45;8;29;49;83;6;46;36...,46;19;13;29,NaN,NaN,52;100;72;131;116;11;71;12;8;113;28;73;6;132;9...,664359;276966;734911;103617;562294,11395;79112;115065;77033;36176,NaN,9826;105;8525;5488;7281,9708;5553;6745;7477;7150,NaN,NaN,NaN,3;1,2,1,NaN
1,1184123,2,1,5;13,2,1,803.0,75;29,33,NaN,NaN,NaN,338851;361151;542834;496283;229952,80263;39618;53539;180;38163,NaN,4391;9140;5669;1348;4388,9401;7724;1380;8890;7153,NaN,NaN,NaN,3;1,1,1,NaN
2,76072711,1,1,13;10,5,1,927.0,70;12;28;106;59;49;41;6;42;115;35;116;36;11;96,51;22;79;81;70;6;21;4;41;35,NaN,NaN,77;72;80;116;101;13;1;109;8;50;6;42;76;9;46;36...,746140;695808;126355;771775;411511,105115;71378;41409;74061;44005,NaN,1502;5488;9826;2187;8088,5005;9154;2756;5612;4209,NaN,NaN,NaN,3;1,1,3,NaN
3,63071413,4,1,11,7,1,486.0,93;70;77;47;76;75;69;29;59;49;122;42;46;23;17;...,24;70;19;13;30;10,NaN,NaN,100;80;92;37;116;13;47;4;71;8;50;28;98;115;6;4...,283399;402245;734509;654027;32061,74010;32918;67882;116802;20957,NaN,1619;7342;3064;9213;8525,810;2438;5659;1844;9262,NaN,NaN,NaN,1;3,1;2,3,NaN
4,81294159,5,1,11,6,2,112.0,75;29;49;122;36;11,NaN,NaN,NaN,131;116;13;8;6;132;42;9;59;18;58;64;129;103,562294;157603;589706;657719;495672,62764;97803;89066;55545;74061,NaN,477;9826;5808;644;2747,5483;2199;5424;1511;7751,NaN,NaN,NaN,3;1;4;2,1,3,NaN


(8798814, 3)

,aid,uid,label
0,699,78508957,-1
1,1991,3637295,-1
2,1119,19229018,-1
3,2013,79277120,-1
4,692,41528441,-1


(2265989, 2)

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [3]:
ad.groupby('productType').aid.count()

productType
4     55
6     33
9     41
11    44
Name: aid, dtype: int64

In [4]:
ad = ad.sort_values(by='aid')

ad.head()

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
142,6,7565,766460,1702986,35,94,0,4
120,7,3387,163957,221432,42,137,3826,11
149,12,388,420987,1612095,35,67,113,9
137,18,8203,74452,857791,95,218,0,6
152,70,327,5616,5977,22,27,113,9


In [5]:
ad['count'] = list(train.groupby('aid').aid.count())
ad['pos_percent'] = list(train[train.label == 1].groupby('aid').aid.count() / train.groupby('aid').aid.count())

In [6]:
productType_count = ad.groupby(
    ['productType'], as_index=False)['count'].agg({
        'count_sum':
        'sum'
    })
productType_count.head()

,productType,count_sum
0,4,3425170
1,6,1178018
2,9,1645649
3,11,2549977


In [7]:
productType_pos_percent = ad.groupby(
    ['productType'], as_index=False)['pos_percent'].agg({
        'pos_percent_mean':
        'mean'
    })
productType_pos_percent.head()

,productType,pos_percent_mean
0,4,0.047683
1,6,0.047451
2,9,0.048014
3,11,0.047945


In [3]:
split_col = ['marriageStatus', 'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
            'topic1', 'topic2', 'topic3','appIdInstall', 'appIdAction', 'ct', 'os']

In [10]:
def spl_col(x):
    if (type(x) == float) | (type(x) == int):
        return x
    elif type(x) == list:
        return x[0].split(';')
    else:
        return x.split(';')

# for i in tqdm(split_col):
#     user[i] = user[i].apply(spl_col)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.model_selection import StratifiedKFold

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# 生切
percent = 0.8

his = pd.DataFrame()
tr = pd.DataFrame()

for i in ad.aid.tolist():
    tmp = train[train.aid == i]
    n = tmp.shape[0]
    his = pd.concat([his, tmp.iloc[:int(n * percent), :]])
    tr = pd.concat([tr, tmp.iloc[int(n * percent):, :]])
    
his.shape[0] + tr.shape[0]
his.shape[0] / tr.shape[0]

In [ ]:
# kflod
kf = KFold(train.shape[0], n_folds=5, random_state=201803)

his = pd.DataFrame()
tr = pd.DataFrame()

for i, (his_idx, tr_idx) in enumerate(kf):
    his = train.iloc[his_idx]
    tr = train.iloc[tr_idx]
    break
        
his.shape[0] + tr.shape[0]
his.shape[0] / tr.shape[0]

In [11]:
# StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=201803)

his_aid = pd.DataFrame()
tr_aid = pd.DataFrame()

for i in ad.aid:
    tmp = train[train.aid == i]
    for his_idx, tr_idx in skf.split(tmp, tmp.label):
        tmp_his = tmp.iloc[his_idx]
        tmp_tr = tmp.iloc[tr_idx]
        break
    his_aid = pd.concat([his_aid, tmp_his])
    tr_aid = pd.concat([tr_aid, tmp_tr])
    
his_aid.shape[0] + tr_aid.shape[0]
his_aid.shape[0] / tr_aid.shape[0]

# his_pid = pd.DataFrame()
# tr_pid = pd.DataFrame()

# for i in tqdm(ad.productId):
#     tmp = train[train.aid == i]
#     for his_idx, tr_idx in skf.split(tmp, tmp.label):
#         tmp_his = tmp.iloc[his_idx]
#         tmp_tr = tmp.iloc[tr_idx]
#         break
#     his_pid = pd.concat([his_pid, tmp_his])
#     tr_pid = pd.concat([tr_pid, tmp_tr])
    
# his_pid.shape[0] + tr_pid.shape[0]
# his_pid.shape[0] / tr_pid.shape[0]

8798814

3.999598841755005

In [22]:
# his.shape
# his.head()

his[['aid', 'uid']].to_csv('output/his_auid.csv', index=None)

In [17]:
ad['his_aid_pos_percent'] = list(his_aid[his_aid.label == 1].groupby('aid').aid.count() / his.groupby('aid').aid.count())
ad['aid_his_tr_minus'] = abs(ad['his_aid_pos_percent'] - ad['pos_percent'])
ad[['pos_percent', 'his_aid_pos_percent', 'aid_his_tr_minus']].sort_values(by=['aid_his_tr_minus'])[-5:]

KeyboardInterrupt: 

In [12]:
his = his_aid
his_pos = his[his.label == 1]

# his = his.merge(user, on=['uid'], how='left')
his_pos = his_pos.merge(user, on=['uid'], how='left')

# his.shape
# his.head()
his_pos.shape
his_pos.head()

(337495, 26)

,aid,uid,label,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,6,55084009,1,2,2,10,6,2,346.0,70;86;118;109;47;76;48;28;8;83;108;6;2;39;104;...,1;46;70;19;13;37;72,NaN,NaN,52;92;131;116;53;47;78;128;8;30;6;132;42;43;76...,71200;93317;87894;8561;698194,107626;99039;49117;52864;461,NaN,8850;9826;9394;7037;3114,313;8543;1470;3600;5509,NaN,NaN,NaN,3;1,1,1,NaN
1,6,30475717,1,2,1,11,7,2,783.0,70;86;37;12;98;75;8;29;49;83;6;2;42;46;115;88;...,58;73;9;31;70;28;3;44;65,NaN,NaN,NaN,528450;478647;319159;761845;530883,101564;4837;38532;34154;110188,NaN,9826;7684;5382;9205;9423,2913;5481;9880;9751;9205,NaN,NaN,NaN,3,2,0,NaN
2,6,15398897,1,1,2,5;13,1,1,812.0,70;109;100;47;76;28;106;6;5;17;9;56;116;36;11,58;21;49,NaN,NaN,52;131;116;105;78;71;8;30;28;98;73;6;5;76;121;...,788904;372067;222150;322823;410278,9279;60463;103645;105115;6690,NaN,1978;4301;1767;934;3362,4947;9705;506;8147;1806,NaN,NaN,NaN,1;3,1,3,NaN
3,6,55825267,1,2,2,10,7,1,737.0,70;28;59;49;15;6;57;36;11,1;70;54,NaN,NaN,72;92;78;109;8;30;28;6;42;43;55;27;75;58;64;90...,627369;338654;336945;288185;429629,75516;79676;106194;7234;66915,NaN,2802;9826;8782;7;704,9204;7599;5260;7350;1632,NaN,NaN,NaN,3;4;1,1,3,1.0
4,6,81699849,1,1,2,6;13,2,1,346.0,70;76;69;33;29;36;11;54,22;73;1;52;46;70;21;19;4;28;3;18;13;65;14;27;29,4;9;1;5;2,NaN,37;116;134;71;109;6;132;76;27;46;121;129;114;4...,362980;682774;98364;2268;60190,44452;56906;74860;95597;34577,30214;36499;36983;8820;42833,7433;8562;5199;469;9499,8462;7459;9751;8950;5095,1614;5918;644;7594;8720,NaN,NaN,3;1,1,1,NaN


In [13]:
# for i in tqdm(split_col):
#     his[i] = his[i].apply(spl_col)
    
# his.shape
# his.head()

for i in tqdm(split_col):
    his_pos[i] = his_pos[i].apply(spl_col)
    
his_pos.shape
his_pos.head()

100%|██████████| 16/16 [00:25<00:00,  1.58s/it]


(337495, 26)

,aid,uid,label,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,6,55084009,1,2,2,[10],6,2,346.0,"[70, 86, 118, 109, 47, 76, 48, 28, 8, 83, 108,...","[1, 46, 70, 19, 13, 37, 72]",NaN,NaN,"[52, 92, 131, 116, 53, 47, 78, 128, 8, 30, 6, ...","[71200, 93317, 87894, 8561, 698194]","[107626, 99039, 49117, 52864, 461]",NaN,"[8850, 9826, 9394, 7037, 3114]","[313, 8543, 1470, 3600, 5509]",NaN,NaN,NaN,"[3, 1]",[1],1,NaN
1,6,30475717,1,2,1,[11],7,2,783.0,"[70, 86, 37, 12, 98, 75, 8, 29, 49, 83, 6, 2, ...","[58, 73, 9, 31, 70, 28, 3, 44, 65]",NaN,NaN,NaN,"[528450, 478647, 319159, 761845, 530883]","[101564, 4837, 38532, 34154, 110188]",NaN,"[9826, 7684, 5382, 9205, 9423]","[2913, 5481, 9880, 9751, 9205]",NaN,NaN,NaN,[3],[2],0,NaN
2,6,15398897,1,1,2,"[5, 13]",1,1,812.0,"[70, 109, 100, 47, 76, 28, 106, 6, 5, 17, 9, 5...","[58, 21, 49]",NaN,NaN,"[52, 131, 116, 105, 78, 71, 8, 30, 28, 98, 73,...","[788904, 372067, 222150, 322823, 410278]","[9279, 60463, 103645, 105115, 6690]",NaN,"[1978, 4301, 1767, 934, 3362]","[4947, 9705, 506, 8147, 1806]",NaN,NaN,NaN,"[1, 3]",[1],3,NaN
3,6,55825267,1,2,2,[10],7,1,737.0,"[70, 28, 59, 49, 15, 6, 57, 36, 11]","[1, 70, 54]",NaN,NaN,"[72, 92, 78, 109, 8, 30, 28, 6, 42, 43, 55, 27...","[627369, 338654, 336945, 288185, 429629]","[75516, 79676, 106194, 7234, 66915]",NaN,"[2802, 9826, 8782, 7, 704]","[9204, 7599, 5260, 7350, 1632]",NaN,NaN,NaN,"[3, 4, 1]",[1],3,1.0
4,6,81699849,1,1,2,"[6, 13]",2,1,346.0,"[70, 76, 69, 33, 29, 36, 11, 54]","[22, 73, 1, 52, 46, 70, 21, 19, 4, 28, 3, 18, ...","[4, 9, 1, 5, 2]",NaN,"[37, 116, 134, 71, 109, 6, 132, 76, 27, 46, 12...","[362980, 682774, 98364, 2268, 60190]","[44452, 56906, 74860, 95597, 34577]","[30214, 36499, 36983, 8820, 42833]","[7433, 8562, 5199, 469, 9499]","[8462, 7459, 9751, 8950, 5095]","[1614, 5918, 644, 7594, 8720]",NaN,NaN,"[3, 1]",[1],1,NaN


In [14]:
from collections import defaultdict, Counter
import ast

In [4]:
def save_obj(obj, name):
    with open('output/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('output/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
marriageStatus_dict = defaultdict(lambda: 0)
interest1_dict = defaultdict(lambda: 0)
interest2_dict = defaultdict(lambda: 0)
interest3_dict = defaultdict(lambda: 0)
interest4_dict = defaultdict(lambda: 0)
interest5_dict = defaultdict(lambda: 0)
kw1_dict = defaultdict(lambda: 0)
kw2_dict = defaultdict(lambda: 0)
kw3_dict = defaultdict(lambda: 0)
topic1_dict = defaultdict(lambda: 0)
topic2_dict = defaultdict(lambda: 0)
topic3_dict = defaultdict(lambda: 0)
appIdInstall_dict = defaultdict(lambda: 0)
appIdAction_dict = defaultdict(lambda: 0)
ct_dict = defaultdict(lambda: 0)
os_dict = defaultdict(lambda: 0)

for j in range(0, his_pos.shape[0]):
    for i in split_col:
        if type(his_pos[i][j]) == list:
            for word in his_pos[i][j]:
                locals()[i + '_dict'][word] += 1
    if j == his_pos.shape[0]-1:
        for i in split_col:
            tmp_dict = ast.literal_eval(str(Counter(locals()[i + '_dict']))[8:-1])
            save_obj(tmp_dict, str(i + '_pos_dict'))

In [5]:
for i in split_col:
    locals()[i + '_dict'] = load_obj(str(i + '_pos_dict'))
    print(str(i + '_dict') + ':' + str(len(locals()[i + '_dict'])))

marriageStatus_dict:12
interest1_dict:121
interest2_dict:80
interest3_dict:10
interest4_dict:10
interest5_dict:136
kw1_dict:96644
kw2_dict:22787
kw3_dict:4158
topic1_dict:9892
topic2_dict:9435
topic3_dict:3022
appIdInstall_dict:40552
appIdAction_dict:3899
ct_dict:5
os_dict:3


In [80]:
his_label = pd.read_csv('output/his_auid.csv')
his_label['is_his'] = 1

train = train.merge(his_label, on=['aid', 'uid'], how='left')

def is_his_label(x):
    if x == 1:
        return x
    else:
        return 0
train['is_his'] = train['is_his'].apply(is_his_label)

In [82]:
test['label'] = -99
test['is_his'] = -99

df = pd.concat([train, test])
df.shape

(11064803, 4)

In [2]:
df = pd.read_csv('df.csv')

# df.columns = ['index', 'aid', 'uid', 'label', 'is_his']
# df.shape
# df.head()

# df.to_csv('df.csv', index=None)

In [28]:
tr = df[df.is_his != 1]
tr.shape

tr = tr.merge(user, on=['uid'], how='left')
for i in tqdm(split_col):
    tr[i] = tr[i].apply(spl_col)

for i in tqdm(split_col):
    tmp_list = list(np.zeros(tr.shape[0]))
    for k in range(0, tr.shape[0]):
        if type(tr[i][k]) == list:
            for j in tr[i][k]:
                tmp_list[k] = tmp_list[k] + locals()[i + '_dict'][j]
        tr[str(i + 'count_score')] = tmp_list


  0%|          | 0/16 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [24]:
len(tmp_list)

4025893

In [ ]:
tr.to_csv('tr_processing.csv', index=None)